In [2]:
"""

Compute Bayesian weights based on Davison et al. 23 reference

"""

'\n\nCompute Bayesian weights based on Davison et al. 23 reference\n\n'

In [1]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf
import os

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-burgardc'


READ IN DATA

In [4]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
outputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_raw = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'

In [5]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [6]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

REFERENCE

In [7]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
davison_file = xr.open_dataset(inputpath_davison + 'steadystate_davison23.nc')

In [8]:
mass_balance_davison = davison_file['basal_melt_obs'] - davison_file['SMB_obs'] + davison_file['calving_obs']
mass_balance_davison_unc = np.sqrt(davison_file['basal_melt_unc']**2 + davison_file['SMB_unc']**2 + davison_file['calving_obs']**2)

SMB

In [9]:
inputpath_atmo_Chris = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_atmo_Nico = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SMB_EMULATED/'
scenario = 'historical'

melt_atmo_list = []
for mod in ['CESM2','CNRM-CM6-1','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']:
    if os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc'):
        melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        melt_atmo['time'] = melt_atmo['time'].dt.year
        melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
    else:
        melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
        
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2-WACCM','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 
           'MRI-ESM2-0']:
    melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
    melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
    
melt_atmo_all = xr.concat(melt_atmo_list, dim='model')

BASAL MELT

In [10]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

In [11]:
## Melt outputpath
Gt_allmod_list = []
box1_allmod_list = []

scenario = 'historical'
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']: #'CNRM-CM6-1',
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list = []
    for mparam in param_classic_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']

    melt1D_list = []
    for mparam in param_NN_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    
    Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

Gt_allmod = xr.concat(Gt_allmod_list, dim='model')
    

GL FLUX FOR ABUMIP

In [12]:
GL_flux_ABUMIP = xr.open_dataset(outputpath_GL + 'all_GL_fluxes.nc')

SUM OF ALL TERMS

In [13]:
mass_balance_simu = Gt_allmod - melt_atmo_all + davison_file['calving_obs']

In [14]:
diff_mod_obs = mass_balance_simu.sel(time=range(1980,2009)).mean('time') - mass_balance_davison
sigma_obs = mass_balance_davison_unc
sigma_mod = sigma_obs * 10

In [15]:
s_j = np.exp(-((diff_mod_obs)**2/(sigma_obs**2 + sigma_mod**2)))

weight = (s_j / (s_j.sum(['model','param'])))

weight_clean = weight#.where(np.isfinite(weight),0)
weight_clean.to_dataset(name='bay_weights').to_netcdf(outputpath_weights + 'bayesian_weights_davison.nc')

In [16]:
ensmean_weighted = uf.weighted_mean(mass_balance_simu, ['model','param'], weight_clean)
ensmean_nonweighted = mass_balance_simu.mean(['model','param'])

da_weighted = mass_balance_simu.weighted(weight_clean)

ensstat_nonweighted = mass_balance_simu.quantile([0.05,0.5,0.95],dim=['model','param'])
ensstat_weighted = da_weighted.quantile([0.05,0.5,0.95],dim=['model','param'])

In [17]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison.sel(Nisf=kisf).values
    melt_obs_unc_kisf = mass_balance_davison_unc.sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=ensstat_weighted.time, y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    
    ax[i].plot(ensstat_weighted.time, ensstat_weighted.sel(quantile=0.5,Nisf=kisf), color='blue')
    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_weighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,Nisf=kisf), color='blue',alpha=0.2)
    #ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_nonweighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_nonweighted.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',alpha=0.2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'ensmean_weighted_allmod_allmassbalance_anoISMIP.pdf')

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


  0%|          | 0/64 [00:00<?, ?it/s]

In [20]:

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison.sel(Nisf=kisf).values
    melt_obs_unc_kisf = mass_balance_davison_unc.sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=ensstat_weighted.time, y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    
    ax[i].plot(ensstat_weighted.time, ensstat_weighted.sel(quantile=0.5,Nisf=kisf), color='blue')
    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_weighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,Nisf=kisf), color='blue',alpha=0.2)
    #ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_nonweighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_nonweighted.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',alpha=0.2)
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ref'].sel(Nisf=kisf), color='orange')
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='red')


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'ensmean_weighted_allmod_allmassbalance_withABUMIPlimit_anoISMIP.pdf')

  0%|          | 0/64 [00:00<?, ?it/s]